# Standard survey 

The same total count is used in...

## RAG summary: Introduction 

"At your school, a total of [number] pupils took part in the #BeeWell survey. This section shows how the answers of pupils at your school compare with pupils from other schools in Northern Devon."

## RAG summary: Understanding topic scores

"Please note that although a total of [number] pupils from your school took part, the RAG ratings given in this section are based only on responses from pupils who completed all the questions within each topic."

## Who took part?

""There were [number] pupils at your school who took part in the #BeeWell survey. This section describes the sample of pupils who completed the survey."

# Standard survey 

## For every topic, "Comparison with other schools"

"Your school had [number] complete responses. Across Northern Devon, there were [number]] complete responses from [number] schools. The average score for the pupils at your school, compared to other schools in Northern Devon, was:

[RAG rating for subgroup]"


* All pupils 
* By year group (Year 8, Year 10)
* By gender (Boys, Girls)
* By FSM status (non-FSM, FSM)
* By SEN status (non-SEN, SEN)


# Introduction for both STANDARD and SYMBOL (only needs to be generated once)

## About: Who took part in the #BeeWell survey in North Devon?


# Standard
Check these values:

"This year, 1298 pupils in Years 8 and 10 at five secondary schools from across North Devon and Torridge completed the standard version of the #BeeWell survey. 

A symbol version of the survey was also completed by 101 pupils in Years 7 to 11 from two non-mainstream schools in Northern Devon."


# Symbol



In [4]:
import pandas as pd

totals_df = pd.read_csv(
    "../data/real/all_schools_overall_counts.csv",
)

# Filter for rows where all categories are "All"
all_pupils_totals = totals_df[
    (totals_df["year_group_lab"] == "All")
    & (totals_df["fsm_lab"] == "All")
    & (totals_df["sen_lab"] == "All")
    & (totals_df["gender_lab"] == "All")
]

# Calculate the total counts for each survey type
standard_total = all_pupils_totals[all_pupils_totals["survey_type"] == "standard"]["count"].dropna().sum()
symbol_total = all_pupils_totals[all_pupils_totals["survey_type"] == "symbol"]["count"].dropna().sum()

# Count unique schools for each survey type
standard_schools = all_pupils_totals[all_pupils_totals["survey_type"] == "standard"]["school_lab"].nunique()
symbol_schools = all_pupils_totals[all_pupils_totals["survey_type"] == "symbol"]["school_lab"].nunique()

# Preparing the output statement
output_statement = (
    f"This year, {int(standard_total)} pupils in Years 8 and 10 at {standard_schools} secondary schools from across "
    "North Devon and Torridge completed the standard version of the #BeeWell survey. A symbol version of the survey "
    f"was also completed by {int(symbol_total)} pupils in Years 7 to 11 from {symbol_schools} non-mainstream schools "
    "in Northern Devon."
)

print(output_statement)


This year, 1301 pupils in Years 8 and 10 at 5 secondary schools from across North Devon and Torridge completed the standard version of the #BeeWell survey. A symbol version of the survey was also completed by 101 pupils in Years 7 to 11 from 2 non-mainstream schools in Northern Devon.


# Symbol survey




## Who took part?

"There were 30 pupils at your school who took part in the #BeeWell survey. This page describes the sample of pupils who completed the survey."